In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from timeit import default_timer as timer  
from numba import jit, cuda

In [2]:
df = pd.read_csv(r'us_accidents_weather_data_CLEAN.csv')
#data = df[['ID', 'Severity', 'Weather_Timestamp', 'Temperature(F)','Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)','Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition','Sunrise_Sunset']]
data = df[['Severity', 'Temperature(F)','Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)','Wind_Speed(mph)', 'Precipitation(in)']]
data = data.dropna()
data = data[data['Severity'] == 4]
#data = data.sample(200000)
data = data.reset_index(drop=True)
print(len(data))
print(data.head(5))
k = 5
eps = 5
iterationsMax = 20
clusters = pd.DataFrame(data = 0.0, columns=data.columns, index=range(k))
c = np.full(len(data), -1)

128167
   Severity  Temperature(F)  Wind_Chill(F)  Humidity(%)  Pressure(in)  \
0         4            33.4           26.6         79.0         29.62   
1         4            23.0           12.4         86.0         29.68   
2         4            26.6           16.8         86.0         29.68   
3         4            19.9            7.3         81.0         29.85   
4         4            19.9            7.3         81.0         29.85   

   Visibility(mi)  Wind_Speed(mph)  Precipitation(in)  
0            10.0              8.1                0.0  
1             1.5             10.4                0.0  
2             2.0             10.4                0.0  
3             2.0             12.7                0.0  
4             2.0             12.7                0.0  


In [3]:
# data: points df
# clusters: clusters df
@jit(target_backend='cuda', nogil=True, cache=True) 
def initClusters(data, clusters):
    for i, ii in enumerate(clusters.iloc[0]):
        clmax = data.max().iloc[i]
        clmin = data.min().iloc[i]
        for j,jj in clusters.iterrows():
            clusters.iloc[j].iloc[i] = (clmax - clmin) * np.random.random() + clmin

initClusters(data, clusters)
print(clusters)

C:\Users\justi\AppData\Local\Temp\ipykernel_8332\3457171471.py:3: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda', nogil=True, cache=True)
C:\Users\justi\AppData\Local\Temp\ipykernel_8332\3457171471.py:3: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "initClusters" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\justi\AppData\Local\Temp\ipykernel_8332\3457171471.py (3)

File "..\..\..\..\AppData\Local\Temp\ipykernel_8332\3457171471.py", line 3:
<source missing, REPL/exec in use?>

  @jit(target_backend='cuda', nogil=True, cache=True)
C:\Users\justi\A

   Severity  Temperature(F)  Wind_Chill(F)  Humidity(%)  Pressure(in)  \
0       4.0      -31.936063     102.899774    82.883503     21.379646   
1       4.0       -4.620233      19.721220    55.874496     26.920071   
2       4.0       53.507572      48.349695    54.330835     27.643691   

   Visibility(mi)  Wind_Speed(mph)  Precipitation(in)  
0       63.552790        38.706244           0.137580  
1       92.998594        71.588796           7.248077  
2       20.393886         4.538672           8.514041  


In [4]:
# p: index of point 
# c: index of cluster
# clusters: clusters df
# data: points df
@jit(target_backend='cuda', nogil=True, cache=True) 
def sse(p: int, c: int, clusters: pd.DataFrame, data: pd.DataFrame) -> float:
    error: float = 0.0
    for i, ii in enumerate(clusters):
        error += (clusters.iloc[c][i] - data.iloc[p][i])**2
    return error


sse(0,0,clusters,data)


C:\Users\justi\AppData\Local\Temp\ipykernel_8332\2454418393.py:5: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda', nogil=True, cache=True)
C:\Users\justi\AppData\Local\Temp\ipykernel_8332\2454418393.py:5: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "sse" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\justi\AppData\Local\Temp\ipykernel_8332\2454418393.py (5)

File "..\..\..\..\AppData\Local\Temp\ipykernel_8332\2454418393.py", line 5:
<source missing, REPL/exec in use?>

  @jit(target_backend='cuda', nogil=True, cache=True)
C:\Users\justi\AppData\Lo

13978.104026502746

In [5]:

# clusters: clusters df
# data: points df
# c: the cluster assignment array
@jit(target_backend='cuda') 
def reassignPoints(data, clusters, c):
    for i,ii in data.iterrows():
        bestIndex = -1
        bestSSE = 100000000
        for j,jj in clusters.iterrows():
            currentSSE = sse(i, j, clusters, data)
            if(currentSSE < bestSSE):
                bestSSE = currentSSE
                bestIndex = j
        c[i] = bestIndex

reassignPoints(data, clusters, c)

C:\Users\justi\AppData\Local\Temp\ipykernel_8332\68328681.py:4: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_8332\68328681.py:4: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "reassignPoints" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\justi\AppData\Local\Temp\ipykernel_8332\68328681.py (4)

File "..\..\..\..\AppData\Local\Temp\ipykernel_8332\68328681.py", line 4:
<source missing, REPL/exec in use?>

  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_8332\68328681.py:4: NumbaW

In [6]:
# clusters: clusters df
# data: points df
# c: the cluster assignment array
@jit(target_backend='cuda') 
def totalError(c, clusters, data):
    error = 0.0
    for i in range(len(c)):
        error = error + sse(i, c[i], clusters, data)
    return(error)

totalError(c, clusters, data)

C:\Users\justi\AppData\Local\Temp\ipykernel_8332\1850685268.py:4: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_8332\1850685268.py:4: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "totalError" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\justi\AppData\Local\Temp\ipykernel_8332\1850685268.py (4)

File "..\..\..\..\AppData\Local\Temp\ipykernel_8332\1850685268.py", line 4:
<source missing, REPL/exec in use?>

  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_8332\1850685268.py:4: 

244052254.29884943

In [7]:
@jit(target_backend='cuda') 
def clusterContent(c, t):
    total = 0
    for i in range(len(c)):
        if (c[i] == t):
            total = total + 1
    return (total)

clusterContent(c, 0)


C:\Users\justi\AppData\Local\Temp\ipykernel_8332\845319178.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda')


6

In [8]:
# clusters: clusters df
# data: points df
# c: the cluster assignment array
# cluster: the focal cluster
@jit(target_backend='cuda') 
def clusterError(c, cluster, clusters, data):
    error = 0.0
    for i in range(len(c)):
        if(c[i] == cluster):
            error = error + sse(i, c[i], clusters, data)
    return(error)

clusterError(c, 0, clusters, data)

C:\Users\justi\AppData\Local\Temp\ipykernel_8332\4154215476.py:5: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_8332\4154215476.py:5: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "clusterError" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\justi\AppData\Local\Temp\ipykernel_8332\4154215476.py (5)

File "..\..\..\..\AppData\Local\Temp\ipykernel_8332\4154215476.py", line 5:
<source missing, REPL/exec in use?>

  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_8332\4154215476.py:5

30118.418357544208

In [9]:
@jit(target_backend='cuda') 
def moveClusters(c, clusters, data):
    for i,ii in clusters.iterrows():
        for j,jj in enumerate(clusters.iloc[i]):
            sum = 0.0
            totP = 0.0
            for k in range(len(c)):
                if (c[k] == i):
                    sum = sum + data.iloc[k][j]
                    totP = totP + 1.0
            if (totP != 0):
                clusters.iloc[i][j] = sum/totP

print(clusters)
moveClusters(c, clusters, data)
print(clusters)

C:\Users\justi\AppData\Local\Temp\ipykernel_8332\2903512841.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_8332\2903512841.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "moveClusters" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\justi\AppData\Local\Temp\ipykernel_8332\2903512841.py (1)

File "..\..\..\..\AppData\Local\Temp\ipykernel_8332\2903512841.py", line 1:
<source missing, REPL/exec in use?>

  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_8332\2903512841.py:1

   Severity  Temperature(F)  Wind_Chill(F)  Humidity(%)  Pressure(in)  \
0       4.0      -31.936063     102.899774    82.883503     21.379646   
1       4.0       -4.620233      19.721220    55.874496     26.920071   
2       4.0       53.507572      48.349695    54.330835     27.643691   

   Visibility(mi)  Wind_Speed(mph)  Precipitation(in)  
0       63.552790        38.706244           0.137580  
1       92.998594        71.588796           7.248077  
2       20.393886         4.538672           8.514041  


c:\Users\justi\AppData\Local\Programs\Python\Python311\Lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "moveClusters" was compiled in object mode without forceobj=True.

File "..\..\..\..\AppData\Local\Temp\ipykernel_8332\2903512841.py", line 3:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaWarning(warn_msg,
c:\Users\justi\AppData\Local\Programs\Python\Python311\Lib\site-packages\numba\core\object_mode_passes.py:161: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected. This is deprecated behaviour that will be removed in Numba 0.59.0.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit

File "..\..\..\..\AppData\Local\Temp\ipykernel_8332\2903512841.py", line 3:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaDeprecationWarning(msg,


   Severity  Temperature(F)  Wind_Chill(F)  Humidity(%)  Pressure(in)  \
0       4.0       -8.900000      88.500000    73.500000     30.335000   
1       4.0       20.266667       2.283333    46.000000     29.830000   
2       4.0       58.297018      54.545020    67.529101     29.602657   

   Visibility(mi)  Wind_Speed(mph)  Precipitation(in)  
0       10.000000         7.000000           0.000000  
1       46.166667       136.750000           0.000000  
2        9.102498         7.887613           0.004984  


In [10]:
@jit(target_backend='cuda') 
def KMeans2(data, k, eps, iterationsMax):
    clusters = pd.DataFrame(data = 0.0, columns=data.columns, index=range(k))
    c = np.full(len(data), -1)
    initClusters(data, clusters)

    oldError = -1 * eps
    newError = 0

    for a in range(1,iterationsMax):
        reassignPoints(data, clusters, c)
        moveClusters(c, clusters, data)
        newError = totalError(c, clusters, data)
        #print(newError)
        if(abs(newError - oldError) < eps):
            break
        oldError = newError
    #print(newError)
    output = clusters
    cle = np.full(k,-1.0)
    ccn = np.full(k, -1)
    for i in range(k):
        cle[i] = round(clusterError(c, i, clusters, data)/len(data),3)
        ccn[i] = clusterContent(c, i)
    output['AVGerror'] = cle
    output['numOfPoints'] = ccn

    return(output)

start = timer()
result = KMeans2(data, k, eps, iterationsMax)
print(result)
print("without GPU:", timer()-start)   

C:\Users\justi\AppData\Local\Temp\ipykernel_8332\2029953386.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_8332\2029953386.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "KMeans2" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\justi\AppData\Local\Temp\ipykernel_8332\2029953386.py (1)

File "..\..\..\..\AppData\Local\Temp\ipykernel_8332\2029953386.py", line 1:
<source missing, REPL/exec in use?>

  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_8332\2029953386.py:1: Num

   Severity  Temperature(F)  Wind_Chill(F)  Humidity(%)  Pressure(in)  \
0       4.0       37.737259      27.805730    73.170548     29.555184   
1       4.0       63.778886      65.667950    83.593799     29.753122   
2       4.0       72.543874      68.384017    42.902081     29.473279   

   Visibility(mi)  Wind_Speed(mph)  Precipitation(in)  AVGerror  numOfPoints  
0        8.524203         8.568182           0.005218   235.822        40540  
1        8.751896         6.610643           0.008705   165.565        47410  
2       10.104409         8.726031           0.000362   215.296        40217  
without GPU: 5250.0737587
